# Как работает Машина тьюринга

$(x_{\text{current}}, q_{\text{current}}) -> (x_{\text{write}}, \text{direction}, q_{\text{current}})$

In [117]:
def turing_machine(programm: list, tape: list):
    '''
    Начальное состояние "q_0"
    Cимвол для перехода читаем с ленты
    program - программа для машины
    tape - "бесконечная" вправо лента
    '''
    
    x_curr = tape[0]
    q_curr = "q_0"
    
    pos_on_tape = 0
    
    while q_curr != 'q_exit':
        next_info = programm[(x_curr, q_curr)]
        tape[pos_on_tape] = next_info["x_write"]
        
        if next_info["direction"] == "left":
            pos_on_tape-=1
        elif next_info["direction"] == "right":
            pos_on_tape+=1
        elif next_info["direction"] == "stand":
            pos_on_tape += 0
        else:
            raise ValueError
            
        if pos_on_tape < 0:
            raise IndexError
        
        if len(tape)-1 == pos_on_tape:
            tape += [0 for i in range(len(tape))]
        
        x_curr = tape[pos_on_tape]
        q_curr = next_info["q_next"]
        
        
        yield x_curr, q_curr
        
    return tape

In [125]:
programm = {
    (0, "q_0"): 
                {"x_write": -1, 
                 "direction": "stand", 
                 "q_next": "q_exit"
                },
    
    (1, "q_0"): 
                {"x_write": 0, 
                 "direction": "right", 
                 "q_next": "q_search_1_in_second_number"
                },
    
    
    
    (1, "q_search_1_in_second_number"): 
                                    {"x_write": 1, 
                                     "direction": "right", 
                                     "q_next": "q_search_1_in_second_number_first_not_empty"
                                    },
    (0, "q_search_1_in_second_number"): 
                                    {"x_write": 0, 
                                     "direction": "right", 
                                     "q_next": "q_search_1_in_second_number_first_empty"
                                    },
    
    
    
    (1, "q_search_1_in_second_number_first_not_empty"): 
                                    {"x_write": 1, 
                                     "direction": "right", 
                                     "q_next": "q_search_1_in_second_number_first_not_empty"
                                    },
    (0, "q_search_1_in_second_number_first_not_empty"): 
                                    {"x_write": 0, 
                                     "direction": "right", 
                                     "q_next": "q_skip_zeroes_right_first_not_empty"
                                    },
    
    
    
    (0, "q_skip_zeroes_right_first_not_empty"): 
                                    {"x_write": 0, 
                                     "direction": "right", 
                                     "q_next": "q_skip_zeroes_right_first_not_empty"
                                    },
    (1, "q_skip_zeroes_right_first_not_empty"): 
                                    {"x_write": 0, 
                                     "direction": "right", 
                                     "q_next": "q_first_not_empty_check_second_empty"
                                    },
    
    
    
    (1, "q_first_not_empty_check_second_empty"): 
                                    {"x_write": 1, 
                                     "direction": "left", 
                                     "q_next": "q_skip_zeroes_left_second_not_empty"
                                    },
    (0, "q_first_not_empty_check_second_empty"): 
                                    {"x_write": 0, 
                                     "direction": "stand", 
                                     "q_next": "q_exit"
                                    },
    
    
    
    (0, "q_skip_zeroes_left_second_not_empty"): 
                                    {"x_write": 0, 
                                     "direction": "left", 
                                     "q_next": "q_skip_zeroes_left_second_not_empty"
                                    },
    (1, "q_skip_zeroes_left_second_not_empty"): 
                                    {"x_write": 0, 
                                     "direction": "left", 
                                     "q_next": "q_second_not_empty_check_first_empty"
                                    },
    
    
    
    
    (1, "q_second_not_empty_check_first_empty"): 
                                    {"x_write": 1, 
                                     "direction": "right", 
                                     "q_next": "q_skip_zeroes_right_first_not_empty"
                                    },
    (0, "q_second_not_empty_check_first_empty"): 
                                    {"x_write": 1, 
                                     "direction": "stand", 
                                     "q_next": "q_exit"
                                    },
    
    
    
    (0, "q_search_1_in_second_number_first_empty"): 
                                    {"x_write": 0, 
                                     "direction": "right", 
                                     "q_next": "q_skip_zeroes_right_first_empty"
                                    },
    (1, "q_search_1_in_second_number_first_empty"): 
                                    {"x_write": 0, 
                                     "direction": "right", 
                                     "q_next": "q_first_empty_check_second_empty"
                                    },

    
    
    (0, "q_first_empty_check_second_empty"): 
                                    {"x_write": 1, 
                                     "direction": "stand", 
                                     "q_next": "q_exit"
                                    },
    (1, "q_first_empty_check_second_empty"): 
                                    {"x_write": 1, 
                                     "direction": "stand", 
                                     "q_next": "q_exit"
                                    },
}

In [126]:
import random as rnd

In [127]:
rnd.seed(42)

In [128]:
mode = "big_stress_test"

In [129]:
%%time
if mode == "mini_stress_test":
    for i in range(1_000_001):
        len_fir_number = rnd.randint(1, 10)
        len_sec_number = rnd.randint(1, 10)

        tape = "1" * len_fir_number
        tape += "0"
        tape += "1" * len_sec_number
        tape = [int(i) for i in tape]
        
        ans = 0
        if len_fir_number <= len_sec_number:
            ans = 1
        program_log = [i for i in turing_machine(programm, tape)]
        if ans != program_log[-1][0]:
            print(program_log)
            print("TEST:", len_fir_number, len_sec_number)
            print("EXPECTED:", ans)
            print("RECEIVED:", program_log[-1][0])
            
            break
        if i % 100000 == 0:
            print(i)
            
elif mode == "big_stress_test":
    for i in range(10_001):
        len_fir_number = rnd.randint(1, 1000)
        len_sec_number = rnd.randint(1, 1000)

        tape = "1" * len_fir_number
        tape += "0"
        tape += "1" * len_sec_number
        tape = [int(i) for i in tape]
        
        ans = 0
        if len_fir_number <= len_sec_number:
            ans = 1
        program_log = [i for i in turing_machine(programm, tape)]
        if ans != program_log[-1][0]:
            
            print("TEST:", len_fir_number, len_sec_number)
            print("EXPECTED:", ans)
            print("RECEIVED:", program_log[-1][0])
            
            break
        if i % 100 == 0:
            print(i)
            
elif mode == "debug_mode":
    tape = input("Enter Tape ")
    tape = [int(i) for i in tape]
    for curr_info in turing_machine(programm, tape):
        print(curr_info)
        tmp = input()
    

0


KeyboardInterrupt: 